In [ ]:
import numpy as np
import pandas as pd
from warnings import filterwarnings as f_w
f_w('ignore')
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500
from homecredit import HomeCreditDataTable
import gc
from itertools import product
from tqdm import tqdm_notebook
from sklearn.feature_selection import f_classif, mutual_info_classif
from lightgbm import LGBMClassifier
from sklearn.metrics import matthews_corrcoef

In [ ]:
table = 'appl'
locator = pd.read_excel('locator.xlsx', index_col = 0)

In [ ]:
hc = HomeCreditDataTable()
hc.train = pd.read_csv(locator.loc[table, 'initial'].split(', ')[0],
                       index_col = 'SK_ID_CURR')
hc.test = pd.read_csv(locator.loc[table, 'initial'].split(', ')[1],
                      index_col = 'SK_ID_CURR')

In [ ]:
#небольшая чистка данных
hc.data = hc.data.replace({'XNA': np.nan,
                           'Unknown': np.nan})
hc.set_column(hc.data.REGION_RATING_CLIENT_W_CITY.replace({-1: 2}))
hc.set_column(hc.data.DAYS_EMPLOYED.replace({365243: np.nan}))
hc.set_column(hc.data.AMT_INCOME_TOTAL.replace(
    dict.fromkeys(hc.train.AMT_INCOME_TOTAL.nlargest(5).values, 
                  np.nan)))
hc.set_column(hc.data.DAYS_LAST_PHONE_CHANGE.replace({0: np.nan}))

In [ ]:
#удаляем цифровые колонки без вариативности в тесте
#только NAME_CONTRACT_TYPE из объектных колонок имеет в тесте менее 1% вариативность 
flags = hc.data.select_dtypes(exclude=object).columns[hc.data.select_dtypes(exclude=object).nunique().eq(2)]
todrop = hc.data[flags].columns[np.logical_not(hc.test[flags].mean().between(.01, .99))]
hc.data = hc.data.drop(columns = todrop)

In [ ]:
#сколько ежемесячных платежей нужно внести, чтобы возвратить сумму кредита
hc.set_column(hc.data.eval('AMT_CREDIT / AMT_ANNUITY'),
              col_name = 'CREDIT_ANNUITY')

In [ ]:
#какая часть цены покупаемых товаров покрыта кредитом
hc.set_column(hc.data.eval('AMT_CREDIT / AMT_GOODS_PRICE'),
              col_name = 'CREDIT_PRICE')

In [ ]:
#сколько таких товаров можно купить за доход
hc.set_column(hc.data.eval('AMT_GOODS_PRICE / AMT_INCOME_TOTAL'),
              col_name = 'PRICE_INCOME')

In [ ]:
#сколько месяцев надо копить, откладывая ежемесячный платеж, чтобы накопить на товар
hc.set_column(hc.data.eval('AMT_GOODS_PRICE / AMT_ANNUITY'),
              col_name = 'PRICE_ANNUITY')

In [ ]:
#сколько месяцев нужно копить, чтобы набрать сумму кредита
hc.set_column(hc.data.eval('AMT_CREDIT / AMT_INCOME_TOTAL'),
              col_name = 'CREDIT_INCOME')

In [ ]:
#какую долю дохода составляет ежемесячный платеж
hc.set_column(hc.data.eval('AMT_ANNUITY / AMT_INCOME_TOTAL'),
              col_name = 'ANNUITY_INCOME')

In [ ]:
#доход на члена семьи
hc.set_column(hc.data.eval('AMT_INCOME_TOTAL / CNT_FAM_MEMBERS'),
              col_name = 'INCOME_FAMMEM')

In [ ]:
#отношение стажа и возраста
hc.set_column(hc.data.eval('DAYS_EMPLOYED / DAYS_BIRTH'),
              col_name = 'EMPL_BIRTH')

In [ ]:
#рейтинг региона, взвешенный по населению
hc.set_column(hc.data.eval('REGION_RATING_CLIENT * REGION_POPULATION_RELATIVE'),
              col_name = 'RATING_POPULATION')
hc.set_column(hc.data.eval('REGION_RATING_CLIENT_W_CITY * REGION_POPULATION_RELATIVE'),
              col_name = 'RATING_CITY_POPULATION')

In [ ]:
#доля детей в семье
hc.set_column(hc.data.eval('CNT_CHILDREN / CNT_FAM_MEMBERS'),
              col_name = 'CHILDREN_FAMMEM')

In [ ]:
#заменим бесконечность (результат деления на 0 в созданных признаках) на NaN
hc.data = hc.data.replace({np.inf: np.nan, -np.inf: np.nan})

In [ ]:
hc.data = hc.data.rename(columns = {c: '_'.join((table.upper(), c))
                                    for c in hc.data.columns})

In [ ]:
hc.data.to_csv(locator.loc[table, 'feat_eng'])

In [ ]:
hc.train[['TARGET']].to_csv('train.csv')
pd.DataFrame(index = hc.test.index).to_csv('test.csv')